In [4]:
from gensim.models.doc2vec import Doc2Vec
import nltk
from collections import defaultdict
from nltk import tokenize
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import copy

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import svd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
model = Doc2Vec.load('apnews_dbow/doc2vec.bin', mmap=None)

In [5]:
# try simple engish
textfile = "facts.txt"

stemmer = SnowballStemmer("english")

sentences = []
init_sentences = []
words = []
lexemes = []
with open(textfile, encoding='utf-8', mode='r') as f:
    text = f.read().lower()
    
    # lets split text for sentences first
    
    # these 2 parts are the same. Either complex one:
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenize.sent_tokenize(text)
    init_sentences = copy.deepcopy(sentences)
    # let's explode sentences to lexemes
    for i, sentence in enumerate(sentences):
        if not sentence:
            continue
        s_words = [word for word
                    in tokenize.word_tokenize(sentence)
                    if word not in (',', '.', ':', '-', ';', '?', '!', '"', "``", "`", "''")
                ]
        s_lexemes = [stemmer.stem(word) for word in s_words]
        for j, word in enumerate(s_words):
            sentences[i] = sentences[i].replace(word, s_lexemes[j])
        words.append(s_words)
        lexemes.append(s_lexemes)
#flattening lexemes list
lexemes = [item for sublist in lexemes for item in sublist]

In [9]:
#tf-idf vectorization
vectorizer = TfidfVectorizer()
tdm = vectorizer.fit_transform(sentences).toarray()

In [11]:
#PCA
pca = PCA(n_components = 0.8,  svd_solver="full")
tdm_reduced = pca.fit_transform(tdm)

In [12]:
def search_query(query, facts_tdm):
        query = query.lower()
        query_words = [word for word
                    in tokenize.word_tokenize(query)
                    if word not in (',', '.', ':', '-', ';', '?', '!', '"', "``", "`", "''")
                ]
        query_lexemes = [stemmer.stem(word) for word in query_words]
        for j, word in enumerate(query_words):
            query = query.replace(word, query_lexemes[j])
        queries_list = []
        queries_list.append(query)
        query_tdm = vectorizer.transform(queries_list).toarray()
        query_tdm_reduced = pca.transform(query_tdm)
        similarity = cosine_similarity(facts_tdm, query_tdm_reduced).reshape(-1)
        top_matches_idx = np.argsort(similarity)[::-1][:5]
        return [init_sentences[i] for i in top_matches_idx]    

In [13]:
import locale
locale.setlocale(locale.LC_ALL, '')

# Api key found on https://translate.yandex.com/developers/keys
api_key = open("yandex.translate.key").read()   # todo your key in the file

from yandex.Translater import Translater
tr = Translater()

In [19]:
query = "франция"
tr.set_key(api_key)
tr.set_from_lang('ru')
tr.set_to_lang('en')

tr.set_text(query)
result = tr.translate()

matches = search_query(result, tdm_reduced)
print("Found:")
tr.set_from_lang('en')
tr.set_to_lang('ru')
for match in matches:
    tr.set_text(match)
    print(tr.translate())

Found:
146. во Франции, это законно выйти замуж за мертвого человека.
8. в 1386 г. свинья во Франции был казнен путем публичного повешения за убийство ребенка.
140. то, что называется “французским поцелуем” в англоязычных странах известен как “английский поцелуй” во Франции.
1.
86. большинство частиц пыли в вашем доме сделаны из мертвой кожи!
